# 2.9_7 学習と検証の実施 for menzu

- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。
- menzu でやるぞい

# 学習目標

1.	SSDの学習を実装できるようになる

# 事前準備

- AWS EC2 のGPUインスタンスを使用します
- フォルダ「utils」のssd_model.pyをします

In [1]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cpu


# DatasetとDataLoaderを作成する

In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# ファイルパスのリストを取得
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoaderを作成する
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

/Users/shotashimizu/github.com/banbiossa/pytorch_advanced/.venv/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [4]:
%load_ext blackcellmagic
from pathlib import Path

%load_ext autoreload
%autoreload 2

In [5]:
# from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn
from pytorch_advanced.c02_ssd.ssd_model_mob import (
    make_datapath_list_zumen,
    voc_from_anocci_json,
)
from pytorch_advanced.c02_ssd.ssd_model import VOCDataset, DataTransform, od_collate_fn


# ファイルパスのリストを取得
(
    train_img_list_menzu,
    train_anno_list_menzu,
    val_img_list_menzu,
    val_anno_list_menzu,
) = make_datapath_list_zumen(Path("."))

# Datasetを作成
voc_classes_menzu = [
    "menzu",
]
color_mean = (200, 200, 200)
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset_menzu = VOCDataset(
    train_img_list_menzu,
    train_anno_list_menzu,
    phase="train",
    transform=DataTransform(input_size, color_mean),
    transform_anno=voc_from_anocci_json,
)

val_dataset_menzu = VOCDataset(
    val_img_list_menzu,
    val_anno_list_menzu,
    phase="val",
    transform=DataTransform(input_size, color_mean),
    transform_anno=voc_from_anocci_json,
)


# DataLoaderを作成する
batch_size = 32

train_dataloader_menzu = data.DataLoader(
    train_dataset_menzu, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn
)

val_dataloader_menzu = data.DataLoader(
    val_dataset_menzu, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn
)

# 辞書オブジェクトにまとめる
dataloaders_dict_menzu = {"train": train_dataloader_menzu, "val": val_dataloader_menzu}

/Users/shotashimizu/github.com/banbiossa/pytorch_advanced/.venv/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


# ネットワークモデルの作成する

In [6]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    # 'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


使用デバイス： cpu
ネットワーク設定完了：学習済みの重みをロードしました


# 損失関数と最適化手法を定義する

In [7]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 学習・検証を実施する

In [8]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    try:
        for epoch in range(num_epochs+1):

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            print('-------------')
            print('Epoch {}/{}'.format(epoch+1, num_epochs))
            print('-------------')

            # epochごとの訓練と検証のループ
            for phase in ['train', 'val']:
                if phase == 'train':
                    net.train()  # モデルを訓練モードに
                    print('（train）')
                else:
                    if((epoch+1) % 10 == 0):
                        net.eval()   # モデルを検証モードに
                        print('-------------')
                        print('（val）')
                    else:
                        # 検証は10回に1回だけ行う
                        continue

                # データローダーからminibatchずつ取り出すループ
                for images, targets in dataloaders_dict[phase]:

                    # GPUが使えるならGPUにデータを送る
                    images = images.to(device)
                    targets = [ann.to(device)
                               for ann in targets]  # リストの各要素のテンソルをGPUへ

                    # optimizerを初期化
                    optimizer.zero_grad()

                    # 順伝搬（forward）計算
                    with torch.set_grad_enabled(phase == 'train'):
                        # 順伝搬（forward）計算
                        outputs = net(images)

                        # 損失の計算
                        loss_l, loss_c = criterion(outputs, targets)
                        loss = loss_l + loss_c

                        # 訓練時はバックプロパゲーション
                        if phase == 'train':
                            loss.backward()  # 勾配の計算

                            # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                            nn.utils.clip_grad_value_(
                                net.parameters(), clip_value=2.0)

                            optimizer.step()  # パラメータ更新

                            if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                                t_iter_finish = time.time()
                                duration = t_iter_finish - t_iter_start
                                print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                    iteration, loss.item(), duration))
                                t_iter_start = time.time()

                            epoch_train_loss += loss.item()
                            iteration += 1

                        # 検証時
                        else:
                            epoch_val_loss += loss.item()
                            

        # epochのphaseごとのloss （Issue158での誤植修正）
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')
            
    except IndexError as e:
        print(e)
        return locals() 

In [9]:
# 学習・検証を実行する
num_epochs= 50  
all_locals = train_model(net, dataloaders_dict_menzu, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cpu
-------------
Epoch 1/50
-------------
（train）


/Users/shotashimizu/github.com/banbiossa/pytorch_advanced/pytorch_advanced/c02_ssd/data_augumentation.py:326: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


イテレーション 10 || Loss: 11.6198 || 10iter: 379.7564 sec.
-------------
Epoch 2/50
-------------
（train）
イテレーション 20 || Loss: 9.9469 || 10iter: 350.4692 sec.
-------------
Epoch 3/50
-------------
（train）


KeyboardInterrupt: 

In [41]:
loc_data, conf_data, dbox_list = all_locals['outputs'] 

In [51]:
# 要素数を把握
num_batch = loc_data.size(0)  # ミニバッチのサイズ
num_dbox = loc_data.size(1)  # DBoxの数 = 8732
num_classes = conf_data.size(2)  # クラス数 = 21

In [52]:
device = "cpu"
jaccard_thresh = 0.5

In [53]:
# 損失の計算に使用するものを格納する変数を作成
# conf_t_label：各DBoxに一番近い正解のBBoxのラベルを格納させる
# loc_t:各DBoxに一番近い正解のBBoxの位置情報を格納させる
conf_t_label = torch.LongTensor(num_batch, num_dbox).to(device)
loc_t = torch.Tensor(num_batch, num_dbox, 4).to(device)

In [56]:
from utils.match import match

In [57]:
# loc_tとconf_t_labelに、
# DBoxと正解アノテーションtargetsをmatchさせた結果を上書きする
for idx in range(num_batch):  # ミニバッチでループ

    # 現在のミニバッチの正解アノテーションのBBoxとラベルを取得
    truths = targets[idx][:, :-1].to(device)  # BBox
    # ラベル [物体1のラベル, 物体2のラベル, …]
    labels = targets[idx][:, -1].to(device)

    # デフォルトボックスを新たな変数で用意
    dbox = dbox_list.to(device)

    # 関数matchを実行し、loc_tとconf_t_labelの内容を更新する
    # （詳細）
    # loc_t:各DBoxに一番近い正解のBBoxの位置情報が上書きされる
    # conf_t_label：各DBoxに一番近いBBoxのラベルが上書きされる
    # ただし、一番近いBBoxとのjaccard overlapが0.5より小さい場合は
    # 正解BBoxのラベルconf_t_labelは背景クラスの0とする
    variance = [0.1, 0.2]
    # このvarianceはDBoxからBBoxに補正計算する際に使用する式の係数です
    match(
        jaccard_thresh,
        truths,
        dbox,
        variance,
        labels,
        loc_t,
        conf_t_label,
        idx,
    )

In [69]:
import os.path as osp

# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET
from itertools import product as product
from math import sqrt as sqrt

import cv2
import numpy as np
import torch

# パッケージのimport
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.utils.data as data
from torch.autograd import Function

In [63]:
# ----------
# 位置の損失：loss_lを計算
# Smooth L1関数で損失を計算する。ただし、物体を発見したDBoxのオフセットのみを計算する
# ----------
# pos_maskをloc_dataのサイズに変形
pos_mask = conf_t_label > 0  # torch.Size([num_batch, 8732])
pos_idx = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)
# Positive DBoxのloc_dataと、教師データloc_tを取得
loc_p = loc_data[pos_idx].view(-1, 4)
loc_t = loc_t[pos_idx].view(-1, 4)

In [68]:
# 物体を発見したPositive DBoxのオフセット情報loc_tの損失（誤差）を計算
loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction="sum")

NameError: name 'F' is not defined

In [50]:
all_locals['criterion']

MultiBoxLoss()

In [72]:
output = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)

In [74]:
output

tensor([[ 1.5228,  0.0301,  0.5960,  0.6422,  2.0850],
        [-2.5129,  0.6893,  0.1608, -1.3404, -0.5606],
        [-2.3963, -1.8834,  0.3018, -1.0998, -0.3744]], requires_grad=True)

In [75]:
target

tensor([[0.1496, 0.0507, 0.1886, 0.2422, 0.3689],
        [0.0485, 0.2835, 0.3535, 0.2574, 0.0571],
        [0.1481, 0.3274, 0.2338, 0.2377, 0.0531]])

In [76]:
loss = F.cross_entropy(output, target)

In [78]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)

In [79]:
input

tensor([[ 0.7232, -2.4441, -1.6124,  0.0986,  0.3389],
        [-0.5117,  0.1218,  0.1301,  0.4398,  0.0342],
        [ 0.8557,  0.8881,  0.5720, -1.5773, -0.0496]], requires_grad=True)

In [80]:
target

tensor([2, 0, 0])

In [81]:
loss

tensor(2.1959, grad_fn=<NllLossBackward0>)

In [77]:
loss

tensor(1.7424, grad_fn=<DivBackward1>)

In [16]:
len(targets)

32

In [18]:
one, two, three = outputs

In [21]:
two.shape

torch.Size([32, 8732, 1])

In [22]:
one.shape

torch.Size([32, 8732, 4])

In [24]:
three.shape

torch.Size([8732, 4])

In [25]:
three

tensor([[0.0133, 0.0133, 0.1000, 0.1000],
        [0.0133, 0.0133, 0.1414, 0.1414],
        [0.0133, 0.0133, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 0.9612, 0.9612],
        [0.5000, 0.5000, 1.0000, 0.6223],
        [0.5000, 0.5000, 0.6223, 1.0000]])

In [ ]:
# 学習・検証を実行する
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

以上